Let $T$ be the flight duration. Let $T_\text{C}$ be the length of a timeslot (short enough that the channel is considered stationary). Then $N = T/T_\text{C}$ is the total number of timeslots over the flight duration. Let $M$ be the number of users that the UAV can serve simulaneously. Then we define $K$, the total number of users, as $M\cdot N$ for convenience.

In [1]:
T = 1000    # seconds
TC = 1      # seconds
N = T // TC # timeslots
M = 1       # users
K = M * N   # users

Ground users are distributed within a dead zone centered $D$ meters away from the base station on the x-axis. For the purpose of this simulation, we consider a circular deadzone of radius $r_\text{D}$. 

In [2]:
D = 5000 # meters
rD = 200 # meters

Here we define a function to randomly generate users within the deadzone

In [3]:
from math import sqrt, cos, sin, pi
import random

def generate_users():
    users = []
    for _ in range(K):
        r = rD * sqrt(random.random())
        theta = random.uniform(0, 2*pi)
        x = r * cos(theta)
        y = r * sin(theta)

        users.append((x, y, 0))
    
    return users

The UAV flies on a circular path of radius $r$ centered $c$ meters away from the base station on the x-axis. These are both parameters we will optimize for.

The circular path is located $H$ meters above the ground. We assume the UAV flies with constant speed $v$.

In [4]:
H = 100 # meters
v = 10  # meters/sec

The UAV’s trajectory can be considered a discrete sequence of positions, each one a function of the flight path radius $r$ and the flight path center point $c$. If the timeslots are indexed $1\dots N$ , then the UAV’s position during timeslot $n$ is given by 
$$\mathbf{w}_n^\text{A}(r,c) = \left[c + r\cos\left(\frac{2\pi n}{N}\right),r\sin\left(\frac{2\pi n}{N}\right),H\right]$$

In [5]:
def wA(n, r, c):
    x = c + r*cos( (v*n*TC)/r )
    y = r*sin( (v*n*TC)/r )
    z = H
    return (x, y, z)

We assume users with a fixed transmission power $P^\text{U}_\text{tx}$. Let $n$ denote the timeslot index with respect to the UAV’s discrete positions along its trajectory, let $\lambda$ denote the wavelength of the signal, and let $G_\text{T}$, $G_\text{R}$ denote the antenna gain at the user and UAV respectively. The power of the signal received by the UAV from the user is then given as
$$P^\text{A}_{n,k} = P^\text{U}_\text{tx}\cdot G_\text{T}G_\text{R}\left(\frac{\lambda}{4\pi \cdot ||\mathbf{w}^\text{A}_n - \mathbf{w}^\text{U}_k||}\right)^2$$

In [6]:
import scipy

PUtx = 1 # watts
GT = 1
GR = 1

FREQ = 2*10**9
WAVELENGTH = scipy.constants.c / FREQ

def PA(n, r, c, wU):
    wAn = wA(n, r, c)

    dist = sqrt((wAn[0] - wU[0])**2
        + (wAn[1] - wU[1])**2
        + (wAn[2] - wU[2])**2)
    
    return PUtx * GT * GR * (WAVELENGTH / (4 * pi * dist))**2

Let $P^\text{A}_\text{tx}$ be the UAV transmission power. It follows that the power of the signal received by the base station during the
$n$th timeslot is expressed
$$P^\text{B}_n(r, c) = P^\text{A}_\text{tx}\cdot G_\text{T}G_\text{R}\left(\frac{\lambda}{4\pi\cdot||\mathbf{w}^\text{A}_n||}\right)^2$$

In [7]:
PAtx = 1 # watts

def PB(n, r, c):
    wAn = wA(n, r, c)

    dist = sqrt(wAn[0]**2 + wAn[1]**2 + wAn[2]**2)

    return PAtx * GT * GR * (WAVELENGTH / (4 * pi * dist))**2

Let $N_0$ be the noise power over the entire bandwidth. In the user-to-UAV link, there are $M$ users transmitting simultaneously to the UAV, the channel for a single user experiences the noise with power $N_0/M$. Thus, the signal-to-noise ratio of the link from the $k$th user to the UAV during the $n$th timeslot can be expressed as
$$\text{SNR}^\text{UA}_{n,k}(r, c) = \frac{P^\text{A}_{n,k}}{N_0/M}$$

In [8]:
from math import log10

BANDWIDTH = 1*10**6 # 1 MHz UAV/BS/GU bandwidth (from energy-efficiency paper)
N0 = 10 ** ( (7.5 - 174 + 10 * log10(BANDWIDTH)) / 10 ) # Noise power in watts

def SNRUA(n, r, c, wU):
    return PA(n, r, c, wU) / (N0 / M)

The signal-to-noise ratio of the UAV-to-BS link during the $n$th timeslot can be expressed
$$\text{SNR}^\text{AB}_n(r, c) = \frac{P^\text{B}_n}{N_0}$$

In [9]:
def SNRAB(n, r, c):
    return PB(n, r, c) / N0

The spectral efficiency of the link from the $k$th user to the UAV during the $n$th timeslot is given by
$$\text{SE}^\text{UA}_{n,k}(r, c) = \log_2\left(1 + \text{SNR}^\text{UA}_{n,k}\right)$$

In [10]:
from math import log2

def SEUA(n, r, c, wU):
    return log2(1 + SNRUA(n, r, c, wU))

The spectral efficiency of the link from the UAV to the base station during the $n$th timeslot is given as
$$\text{SE}^\text{AB}_n(r, c) = \log_2\left(1 + \text{SNR}^\text{AB}_n\right)$$

In [11]:
def SEAB(n, r, c):
    return log2(1 + SNRAB(n, r, c))

We define $\alpha$ as the proportion of each timeslot reserved for user-to-UAV transmission. The proportion of the timeslot reserved for UAV-to-BS transmission is $1-\alpha$. 

The average achievable spectral efficiency of the relay system over the timeslots is then given as
$$\overline{\text{SE}}(\alpha, r, c) = \frac{1}{N}\sum_{n=0}^N \min\left[\alpha \cdot \frac{1}{M}\sum_{k\in\mathcal{U}_n}\text{SE}^\text{UA}_{n,k}\;\mathbf{,}\;(1-\alpha)\cdot\text{SE}^\text{AB}_n\right]$$

In [12]:
def meanSE(alpha, r, c):
    users = generate_users()

    total_se = 0

    for n in range(N):
        ua_se = 0

        for _ in range(M):
            ua_se += SEUA(n, r, c, users.pop())

        ua_se *= (alpha / M)

        ab_se = SEAB(n, r, c)
        ab_se *= (1 - alpha)

        total_se += min(ua_se, ab_se)

    return total_se / N

Optimizing the problem with `scipy.optimize`

In [14]:
from scipy.optimize import minimize_scalar

def objective(r, alpha, c):
    return -meanSE(alpha, r, c)

r_min = 10
r_max = 1000

alpha = 0.5
c = D/2

result = minimize_scalar(objective, bounds=(r_min, r_max), args=(alpha, c), method='bounded')

r_opt = result.x
max_value = -result.fun  # Remember to negate back

print(f"Optimal r: {r_opt}")
print(f"Maximum meanSE: {max_value}")

Optimal r: 601.3985806249214
Maximum meanSE: 0.5141352389808574
